In [1]:
#!pip install tensorflow==2.12    #tensorflow==2.13 or tensorflow==2.12  openCV==4.10.0.84  python==3.9.21


In [2]:
import tensorflow as tf

In [3]:
#!pip install numpy==1.23
#!pip install matplotlib==3.9.4

In [4]:
#!pip install opencv-python

In [5]:
import os
import cv2
import numpy as np

from keras.utils import to_categorical 
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [6]:
#!pip list

In [7]:
dataset_path = "dataset/"

In [8]:
#image parameters
img_size = 128

In [9]:
#prepare data generator
datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.2
                            )
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

Found 3077 images belonging to 3 classes.
Found 769 images belonging to 3 classes.


In [10]:
#Build CNN Mode1
#Define image size(example: 128x128)
img_size = 128
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(img_size, img_size, 3)),
    MaxPooling2D(pool_size=(2,2)),
    
    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(pool_size=(2,2)),
    
    Conv2D(128, (3,3), activation="relu"),
    MaxPooling2D(pool_size=(2,2)),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")     #For binary classification
])

model.compile(loss="binary_crossentropy",
              optimizer=Adam(0.0001),
              metrics=["accuracy"]
             )

model.compile(
    loss="binary_crossentropy",
    optimizer=Adam(0.0001),
    metrics=["accuracy"]
    )

In [ ]:
#Train Model
history = model.fit(
    train_data,epochs=10,
    validation_data=val_data, 
    
)

Epoch 1/10
15/97 [===>..........................] - ETA: 1:10 - loss: -3.1891 - accuracy: 0.4750

C:\Users\Admin\anaconda3\envs\aidl\lib\site-packages\PIL\Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


97/97 [==============================] - 86s 844ms/step - loss: -1253.5253 - accuracy: 0.4956 - val_loss: -8466.3496 - val_accuracy: 0.4980
Epoch 2/10
97/97 [==============================] - 68s 699ms/step - loss: -126495.1172 - accuracy: 0.4982 - val_loss: -442895.9688 - val_accuracy: 0.4980
Epoch 3/10
97/97 [==============================] - 308s 3s/step - loss: -1773960.7500 - accuracy: 0.4982 - val_loss: -4211969.0000 - val_accuracy: 0.4980
Epoch 4/10
97/97 [==============================] - 57s 581ms/step - loss: -9863301.0000 - accuracy: 0.4982 - val_loss: -19100992.0000 - val_accuracy: 0.4980
Epoch 5/10
97/97 [==============================] - 53s 549ms/step - loss: -35316488.0000 - accuracy: 0.4982 - val_loss: -59833020.0000 - val_accuracy: 0.4980
Epoch 6/10
97/97 [==============================] - 58s 596ms/step - loss: -96964152.0000 - accuracy: 0.4982 - val_loss: -149434896.0000 - val_accuracy: 0.4980
Epoch 7/10
64/97 [==================>...........] - ETA: 21s - loss: -189

In [ ]:
#save model
model.save("face_mask_cnn.h5")
print("Model saved as face_mask_cnn.h5")

In [ ]:
#Real-Time Mask Detection
import cv2
import numpy as np
from tensorflow.keras.models import load_model
mask_model = load_model("face_mask_cnn.h5")
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
cap = cv2.VideoCapture(0)
#assuming 'cap' is initialized and'prediction' is defined somewhere
while True:
    
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    
    
    for (x, y, w, h) in faces:
        
        face_roi = frame[y:y+h, x:x+w]
        
        face_roi =cv2.resize(face_roi, (128, 128))
        
        face_roi = face_roi.astype("float32") / 255
        face_roi = np.array([face_roi]) 
        preds = mask_model.predict(face_roi)[0]
        if len(preds) ==2:
            mask, withoutMask = preds
        else:
            mask = preds[0]
            withoutMask = 1 - mask
        if mask > withoutMask:
            
            label = "Mask"
            color = (0, 255, 0)
        else:
            label = "No Mask"
            color = (0, 0, 255)
            
        
            cv2.putText(frame, label, (x, y-10),cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
        
        prediction = model.predict(face_roi)[0]
        
        if prediction < 0.5:
            label = "No Mask"
            color = (0, 0, 255)
        else:
            label = "Mask"
            color = (0, 255, 0)
        
        color = (0, 255, 0) if prediction < 0.5 else (0, 0, 255)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
cv2.putText(frame, label, (x, y-10),
                    
                        
                        
cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            
        
        
cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            
        
face_roi = frame[y:y+h, x:x+w]
        
cv2.imshow("Mask Detection", frame)

        
        
         
key = cv2.waitKey(1) & 0xFF
if key == ord('q'):
    break
                
#cleanup
cap.release()
cv2.destroyAllWindows()